In [1]:
import gurobipy as gp
from gurobipy import *

In [2]:
#For the maximum
class Solution:
    def __init__(self):
        self.lb = -1*float('inf')
        self.node = 1
        self.xrec = []
    def BnB(self, model):
        model.optimize() 
        print('current node number:', str(self.node))
        if model.status == 3 or model.status == 5:
            print('lower bound:', str(self.lb))
            print('prune by infeasible')
            return
        
        ub = int(model.objVal)
        if ub <= self.lb:
            print('lower bound:', str(self.lb))
            print('local upper bound:', str(ub))
            print('prune by bound')
            return
        
        flag = 0
        i = 0
        non_i = 0 # index where val is not int
        for var in model.getVars():
            v = var.x
            if v - int(v) != 0:
                flag = 1
                non_i = i
            i += 1
        if flag == 0:
            tmp_solution = model.objVal
            if tmp_solution > self.lb:
                self.lb = tmp_solution
                self.xrec.clear()
                for v in model.getVars():
                    if v.x != 0:
                        self.xrec.append([v, v.x])
            print('lower bound:', str(self.lb))
            print('local solution:', str(tmp_solution))
            print('Global optimal:', str(self.lb))
            print('prune by optimal')
            return
        else:
            model.addConstr(model.getVars()[non_i] == 1)
            model.update()
            l = model.getConstrs()[-1]
            self.node += 1
            self.BnB(model)
            model.remove(l)
            model.addConstr(model.getVars()[non_i] == 0)
            model.update()
            r = model.getConstrs()[-1]
            self.node += 1
            self.BnB(model)
            model.remove(r)
            return
    
        
        
    

In [3]:
fname = input('Please input flie name correctly:')
m = gp.read(fname)
S = Solution()
S.BnB(m)
print('The optimal is', str(S.lb))
print('The X* are:')
print(S.xrec)
